In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression

from scipy import stats

import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
# Load in full dataset

df = pd.read_csv("housing_full.csv")

In [3]:
# Take subset of useful columns to be used in regression model to reduce clutter

df_subset = df[['Date', 'Zip', 'InventorySeasonallyAdjusted_AllHomes',
             'MedianListingPrice_AllHomes', 'PctOfHomesIncreasingInValues_AllHomes',
             'ZHVI_AllHomes']]

In [4]:
def ols_timeshift(zipcode, df, plots=False):

    # filter full dataset for zipcode of interest, drop null rows
    
    df = df[df['Zip'] == zipcode]
    df.dropna(axis=0, inplace=True)
    
    # drop columns not used in regression model
    
    df_subset = df[['Date', 'Zip', 'InventorySeasonallyAdjusted_AllHomes',
             'MedianListingPrice_AllHomes', 'PctOfHomesIncreasingInValues_AllHomes',
             'ZHVI_AllHomes']]
    
    df_shift = df_subset.copy()
    
    df_date = df[['Date']]
    
    # set up lists that will contain correlation values for each time shift (1-6 months)
    
    season_corrs = []
    zhvi_corrs = []
    pct_corrs = []
    
    # loop through time shifts and append correlation values to lists
    
    for i in range(1, 7):
        df_corrtest = df_subset.copy()
        
        df_corrtest['InventorySeasonallyAdjusted_AllHomes'] = df_corrtest['InventorySeasonallyAdjusted_AllHomes'].shift(i)
        season_corrs.append(stats.pearsonr(df_corrtest['MedianListingPrice_AllHomes'][i:], 
                                           df_corrtest['InventorySeasonallyAdjusted_AllHomes'][i:]))
        
        df_corrtest['PctOfHomesIncreasingInValues_AllHomes'] = df_corrtest['PctOfHomesIncreasingInValues_AllHomes'].shift(i)
        pct_corrs.append(stats.pearsonr(df_corrtest['MedianListingPrice_AllHomes'][i:], 
                                           df_corrtest['PctOfHomesIncreasingInValues_AllHomes'][i:]))
        
        df_corrtest['ZHVI_AllHomes'] = df_corrtest['ZHVI_AllHomes'].shift(i)
        zhvi_corrs.append(stats.pearsonr(df_corrtest['MedianListingPrice_AllHomes'][i:], 
                                         df_corrtest['ZHVI_AllHomes'][i:]))
        
    # find the time shift for each predictor variable that results in best correlation
    # best time shift will be equal to list index + 1 because of 0-indexing
    # some correlations are negative, in which case the min correlation is found
    
    season_shift = season_corrs.index(min(season_corrs))+1
    zhvi_shift = zhvi_corrs.index(max(zhvi_corrs))+1
    pct_shift = pct_corrs.index(max(pct_corrs))+1
    
        
    # shift dataset using optimal time shift settings found in previous step

    df_subset['InventorySeasonallyAdjusted_AllHomes']=df_subset['InventorySeasonallyAdjusted_AllHomes'].shift(season_shift)
    df_subset['ZHVI_AllHomes']=df_subset['ZHVI_AllHomes'].shift(zhvi_shift)
    df_subset['PctOfHomesIncreasingInValues_AllHomes']=df_subset['PctOfHomesIncreasingInValues_AllHomes'].shift(pct_shift)
    
    # subset dataframe into X (independent variables) and Y (dependent variable)
    # also need to account for the shifted variables to make sure dfX and dfY are the same length, so
    # the first x number of rows are removed where x is the max time shift from the 3 independent variables
    
    dfX = df_subset[['InventorySeasonallyAdjusted_AllHomes', 'ZHVI_AllHomes' ,'PctOfHomesIncreasingInValues_AllHomes']][max(season_shift, zhvi_shift, pct_shift):]

    dfY = df_subset[['MedianListingPrice_AllHomes']][max(season_shift, zhvi_shift, pct_shift):]
    
    # split into train/test
    
    length = len(dfX)
    train = round(length*0.8)
    test = length-train

    dfX_train = dfX[:train]
    dfX_test = dfX[-test:]
    dfY_train = dfY[:train]
    dfY_test = dfY[-test:]

    
    # regression model

    regr=sm.OLS(dfY_train, dfX_train).fit()
    
    # predicted values

    y_pred = regr.predict(dfX_test)
    
    dfY_test['predicted'] = y_pred
    
    # return predicted values for insertion into original dataframe
    # indices are preserved for the eventual join

    modeleval = pd.DataFrame()
    modeleval['prediction'] = y_pred
    modeleval['test'] = dfY_test['MedianListingPrice_AllHomes'].values
    modeleval['zhvi'] = dfX_test['ZHVI_AllHomes']

    # optional plot generator
    
    if plots==True:

        modeleval = modeleval.join(df_date)
        plt.scatter(x=modeleval['Date'], y=modeleval['prediction'], label='Predicted')
        plt.plot(modeleval['Date'], modeleval['prediction'])
        plt.scatter(x=modeleval['Date'], y=modeleval['test'], label='Actual')
        plt.plot(modeleval['Date'], modeleval['test'])
        plt.scatter(x=modeleval['Date'], y=modeleval['zhvi'], label='ZHVI')
        plt.plot(modeleval['Date'], modeleval['zhvi'])
        plt.xticks(rotation=90)
        plt.ylabel('Home Price')
        plt.title('Predicted home prices for zip code {}'.format(zipcode))
        plt.grid(which='major', axis='both', alpha=0.5)
        plt.legend()
        plt.show()
        
    # find max time shift from above
 
    shift = max(season_shift, zhvi_shift, pct_shift)
    
    # shift all dependent variables using copy of original dataset
    
    dfX_shift = df_shift[['InventorySeasonallyAdjusted_AllHomes',  'ZHVI_AllHomes', 'PctOfHomesIncreasingInValues_AllHomes']][-shift:]
    
    # depending on max time shift, can use model to forecast into future months
    # e.g. if variables for this zip code were shifted 6 months, we can take
    # the last 6 months' worth of data in the original dataset and project
    # 6 months into the future due to the time lag
    
    forecast_pred = regr.predict(dfX_shift)
    
    forecast_length = len(forecast_pred)
    
    # list of future dates that will be sliced depending on timeshift (6 months is maximum)
    
    date_str = ['2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30']

    # create forecast dataframe containing forecasted values for this zipcode
    
    forecast_df = pd.DataFrame()
    forecast_df['Date'] = date_str[:shift]
    forecast_df['Zip'] = zipcode
    forecast_df['Predicted'] = forecast_pred.values


    return dfY_test['predicted'], forecast_df


In [5]:
# code to loop through every unique zipcode in the original dataset, and create two new dataframes:
# the original dataframe with a 'predicted' column for the portion of the original dataset that was set 
# aside for testing (20%), and a new dataframe with predictions that were forecasted/projected into
# future months for each zipcode

df_pred = pd.Series()
df_forecast = pd.DataFrame()
inc = 0

# NOTE this cell can take ~30min or more to run

for code in df['Zip'].unique():
    try:
        y, f = ols_timeshift(code, df_subset)
        df_pred = pd.concat([df_pred, y])
        df_forecast = pd.concat([df_forecast, f])
        inc += 1
    except:
        print(code)
        inc += 1
    
    # progress indicator
        
    if inc%100 == 0:
        print(inc/len(df['Zip'].unique()))
    

0.005250446287934475
0.01050089257586895
0.015751338863803425
0.0210017851517379
0.026252231439672372
0.03150267772760685
0.03675312401554132
0.0420035703034758
0.04725401659141027
0.052504462879344745
0.05775490916727922
0.0630053554552137
0.06825580174314817
0.07350624803108265
0.07875669431901712
0.0840071406069516
0.08925758689488607
0.09450803318282054
0.09975847947075502
0.10500892575868949
0.11025937204662396
0.11550981833455844
0.12076026462249291
0.1260107109104274
0.13126115719836187
0.13651160348629635
0.14176204977423082
0.1470124960621653
0.15226294235009977
0.15751338863803424
0.16276383492596871
0.1680142812139032
0.17326472750183766
0.17851517378977214
0.1837656200777066
0.18901606636564108
0.19426651265357556
0.19951695894151003
0.2047674052294445
0.21001785151737898
0.21526829780531345
0.22051874409324793
0.2257691903811824
0.23101963666911687
0.23627008295705135
0.24152052924498582
0.2467709755329203
0.2520214218208548
0.25727186810878927
0.26252231439672374
0.267772

In [9]:
df_pred = pd.DataFrame(df_pred)

In [10]:
df = df.join(df_pred)

In [12]:
df.rename(columns={0:"Predicted"}, inplace=True)

In [22]:
df.to_csv(r'output_with_predictions.csv', index=False, header=True)
df_forecast.to_csv(r'forecasted_predictions.csv', index=False, header=True)